In [1]:
from imp import reload
import arm_model
reload(arm_model)
from arm_model import Arm

arm = Arm()
t, y = arm.step()
t,y

C:\Users\CarlF\AppData\Local\Temp\ipykernel_5324\3056382713.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
c:\Users\CarlF\workspace\FTC\tis-but-a-scratch\.venv\lib\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `vetorized`.
  warn("The following arguments have no effect for a chosen solver: {}."


(0.001, -0.001516523705602707)

Wow.  This runs roughly in real-time!


In [2]:
%%time
for i in range(5000):
    arm.step()


CPU times: total: 5.33 s
Wall time: 5.33 s


In [3]:
class PIDControl:
    """So far, it's just a P control
    """
    tgt = 10
    f_range = (-1, 1)
    accumulator = 0.0
    last_err = 0.0

    def __init__(self, p: float =50.0, i:float = 1, d:float = 100):
        self.arm = Arm()
        self.p=p
        self.i=i
        self.d=d

    def clip_f(self, f: float) -> float:
        fmin, fmax = self.f_range
        if f < fmin:
            return fmin
        if f > fmax:
            return fmax
        return f

    def step(self, dt=0.005):
        t,x = self.arm.step(dt)
        err = self.tgt-x 
        delta_err = err - self.last_err
        f = self.clip_f( err * self.p + self.accumulator * self.i + delta_err * self.d)
        self.arm.mot.power = f 

        self.last_err = err
        if abs(err) < 2:
            self.accumulator += err

        return t, x, f 



In [53]:
%load_ext autoreload
%autoreload 2
from arm_graph import output_notebook

output_notebook()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [97]:

from IPython.display import display, HTML
from arm_graph import output_notebook, show, time_vs_angle, time_vs_angle_rate, time_vs_internal_angle, time_vs_power
from bokeh.layouts import row, column

def make_pc(angle):
    pc = PIDControl(.2, 1e-4, 10)
    # Tune I to get roughly the correct oscillation frequency.
    pc.arm.phys.I *= 5
    pc.arm.state=(angle-5, 0, angle-5)
    pc.tgt=angle
    # pc.arm.phys.friction=0.3
    for i in range(3000):
        pc.step(0.001)
    return pc

graph_params = {
    "height":200,
    "width": 500
    }
def show_graph(pc, angle): 
    tva = time_vs_angle(pc.arm, **graph_params)
    tvia = time_vs_internal_angle(pc.arm, fig=tva, color='gray', **graph_params)
    tvp = time_vs_power(pc.arm, **graph_params)
    tvp.x_range=tva.x_range
    display()
    show(column(HTML(f"<h2>Angle: {angle}</h2>"),
    row(tvia, tvp)))
    

angle = 15
show_graph(make_pc(angle), angle)


In [98]:

for angle in range(60, 130, 10):
    pc = make_pc(angle)
    show_graph(pc, angle)
# tvar = time_vs_angle_rate(pc.arm, height=height)
# show(row(tvar, tvp))
